# Get The Latest Calendar Data Past November 19th, 2020
    -The goal of this is to scrape the latest data from the AirBnb website. As of right now, we have data till Nov 19th, 2020.But whenever this function is run we can get the data from Nov 19th, 2020 to whatever exists in the calender.csv. 
    -In the next cell are all the imports that are needed

In [1]:
import numpy as np
import pandas as pd
import requests
import bs4
import csv
from bs4 import BeautifulSoup


In [2]:
# the website we need to access to get the data
bnbdata = "http://insideairbnb.com/get-the-data.html"
# requesting to access the website
page = requests.get(bnbdata)
# using BeautifulSoup package as an HTML  parser
soup = BeautifulSoup(page.content, 'html.parser')


In [3]:
# finding the San Diego data table on the website and creating a list of csv file links
table = soup.find("table", class_="table table-hover table-striped san-diego").find_all('a')

# need to convert the HTML language into just plain links which is stored in results
result = []
for i in table:
    result.append(i.get("href"))


In [4]:
# we need to be able to access the calendar.csv file from the page which will always be in the first index of the list because it would be the most up to date data
latest_calendar_df = pd.read_csv(result[1]) # read in the csv file
print(latest_calendar_df.shape)
latest_calendar_df.head()

# accessing all the listings that we want to find the data for in calendar.csv
listing_df = pd.read_csv(r'data/listings.csv')
listing_df.head()

(5027145, 7)


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (43,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5570,https://www.airbnb.com/rooms/5570,20191121092222,2019-11-21,Ocean front condo on the sand,Missionbeachretreat. Great Ocean Front Condo...,This charming two bedroom beachfront condomini...,Missionbeachretreat. Great Ocean Front Condo...,none,While some may compare San Diego’s Mission Bea...,...,f,f,strict_14_with_grace_period,f,f,2,2,0,0,NaN
1,8488,https://www.airbnb.com/rooms/8488,20191121092222,2019-11-21,Juniper Zen Palace- SD's Best Hood!,Walk every where in San Diegos most charming a...,Private craftsman home in the heart of Histori...,Walk every where in San Diegos most charming a...,none,South Park is a charming community and our nei...,...,f,f,strict_14_with_grace_period,t,t,2,2,0,0,0.36
2,29967,https://www.airbnb.com/rooms/29967,20191121092222,2019-11-21,"Great home, 10 min walk to Beach","This 2 bedroom, 2 full bathroom home offers al...","This two bedroom (1 CA king bed, 1 queen bed),...","This 2 bedroom, 2 full bathroom home offers al...",none,NaN,...,f,f,moderate,f,f,5,5,0,0,0.53
3,38245,https://www.airbnb.com/rooms/38245,20191121092222,2019-11-21,Point Loma/Liberty Station-NTC,Stay in a 1928 Spanish Style home. Den is dow...,Please note: 1928 home; remodeled in 2015. Co...,Stay in a 1928 Spanish Style home. Den is dow...,none,"This neighborhood is an expensive, high-end Po...",...,f,f,moderate,t,t,3,0,3,0,1.28
4,39516,https://www.airbnb.com/rooms/39516,20191121092222,2019-11-21,Art Studio Retreat/Rustic Cabin,We love having explorers from all over the wor...,"Nestled in South/North Park, a gentrified neig...",We love having explorers from all over the wor...,none,"We love our eclectic neighborhood, with a smal...",...,f,f,strict_14_with_grace_period,t,f,1,1,0,0,3.38


In [5]:
# creating a hash set of all the unique ids within the listings.csv
idset = set(listing_df.id.unique())

# deleting all the rows that do not have the listing ids listed in idset
latest_calendar_df = latest_calendar_df[latest_calendar_df.listing_id.isin(idset)]
latest_calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
11680,359971,2020-02-22,f,$96.00,$96.00,28,2000
11681,5570,2020-02-22,f,"$2,050.00","$2,050.00",3,365
11682,5570,2020-02-23,f,"$2,050.00","$2,050.00",3,365
11683,5570,2020-02-24,f,"$2,050.00","$2,050.00",3,365
11684,5570,2020-02-25,f,"$2,050.00","$2,050.00",3,365


In [6]:
# finding all the unique dates that exist in the calendar dataframe
arr2 = latest_calendar_df.date.unique()

# only need data that is past the date of 2020-11-19 so needed to find the index of the '2020-11-20'
index = np.where(arr2 == '2020-11-20')

# making a set with the dates that we will be able to use because it is all the new data we dont have
arr2 = set(arr2[index[0][0]:])

# delete all the rows that do not have the need data
latest_calendar_df = latest_calendar_df[latest_calendar_df.date.isin(arr2)]

In [7]:
print(latest_calendar_df.shape)
# reindex the new dataframe which has all the new data within it
latest_calendar_df = latest_calendar_df.reset_index(drop=True)
latest_calendar_df.head()





(1128183, 7)


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,5570,2020-11-20,t,"$2,050.00","$2,050.00",3,365
1,5570,2020-11-21,f,"$2,050.00","$2,050.00",3,365
2,5570,2020-11-22,f,"$2,050.00","$2,050.00",3,365
3,5570,2020-11-23,f,"$2,050.00","$2,050.00",3,365
4,5570,2020-11-24,f,"$2,050.00","$2,050.00",3,365
